In [1]:
import importlib
import hashlib
from pathlib import Path
from benchmark_utils import render_distribution, visual_benchmark

# Directory with implementations
IMPLS_DIR = Path("impls")

# Cache: module_name -> {"hash": sha1, "result": benchmark_result}
benchmark_cache = {}

In [8]:
# Just restart the notebook and it will run all benchmarks again.
# Cached benchmarks are stored in memory and reused if the source file did not change.


from pprint import pprint


def compute_sha1(path):
    """Return SHA1 hash of the file's contents."""
    with open(path, "rb") as f:
        return hashlib.sha1(f.read()).hexdigest()


def load_module(name):
    """Import or reload impls.<name>."""
    full_name = "impls." + name

    if full_name in globals():
        module = importlib.reload(globals()[full_name])
    else:
        module = importlib.import_module(full_name)
        globals()[full_name] = module

    return module


def report(stats):
    render_distribution(stats)
    pprint(stats["metrics"])


# Iterate over implementation files
for file in IMPLS_DIR.glob("*.py"):
    name = file.stem

    # Skip interface and private modules
    if name == "_interface" or name.startswith("_"):
        continue

    print("Found implementation:", name)

    file_hash = compute_sha1(file)

    # If cached: check whether file changed
    if name in benchmark_cache:
        cached = benchmark_cache[name]
        cached_hash = cached["hash"]
        cached_stats = cached["stats"]

        if cached_hash == file_hash:
            print("  No changes. Using cached benchmark.")
            report(cached_stats)
            continue
        else:
            print("  File changed. Reloading and benchmarking again.")
    else:
        print("  First time seen. Benchmarking.")

    # Load/reload implementation
    module = load_module(name)

    # Require expected constructors
    if not hasattr(module, "producer_constructor") or not hasattr(
        module, "recoverer_constructor"
    ):
        print("  Missing constructors. Skipping.\n")
        continue

    N = 5
    D = 2**20
    producer_constructor = module.producer_constructor
    recoverer_constructor = module.recoverer_constructor
    D = getattr(module, "override_D", lambda _: D)(N)

    # Run benchmark
    stats = visual_benchmark(
        producer_constructor,
        recoverer_constructor,
        N=N,
        D=D,
        passes=10000,
    )

    # Store into cache
    benchmark_cache[name] = {
        "hash": file_hash,
        "stats": stats,
    }

    # Render the new result
    report(stats)

Found implementation: anfext
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 2.9988105750689327,
           'expected_sample_data_size': 16.466615004775722,
           'expected_time_to_recover': 98.8305,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.16189334264220054,
             'bit_redundancy': 414.15250000000003,
             'ideal_bit_efficency': 0.19137647975373717,
             'ideal_bit_redundancy': 338.0242007949139,
             'ideal_packet_efficiency': 0.956882398768686,
             'ideal_packet_redundancy': 67.60484015898277,
             'ideal_time_to_recover': 83.60484015898277,
             'ideal_transmitted_bits': 418.0242007949139,
             'packet_efficiency': 0.8094667132110027,
             'packet_redundancy': 82.8305,
             'permeability': 0.8459416896502878,
             'saturation': 16.0,
             'transmitted_bits': 494.15250000000003}}
Found implementation: anfext2
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 3.0038670068569604,
           'expected_sample_data_size': 16.516823473130284,
           'expected_time_to_recover': 39.623000000000005,
           'packet_size': 5,
           'payload_bits': 80.0},
 'derived': {'bit_efficency': 0.4038058703278399,
             'bit_redundancy': 118.11500000000001,
             'ideal_bit_efficency': 0.47724487832028123,
             'ideal_bit_redundancy': 87.62882879239959,
             'ideal_packet_efficiency': 2.386224391601406,
             'ideal_packet_redundancy': 17.52576575847992,
             'ideal_time_to_recover': 33.52576575847992,
             'ideal_transmitted_bits': 167.6288287923996,
             'packet_efficiency': 2.0190293516391993,
             'packet_redundancy': 23.623000000000005,
             'permeability': 0.8461188137818922,
             'saturation': 16.0,
             'transmitted_bits': 198.115}}
Found implementation: anfext_fullrank
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 3.002207830438622,
           'expected_sample_data_size': 16.473303650279657,
           'expected_time_to_recover': 96.55550000000001,
           'packet_size': 5,
           'payload_bits': 78.25340432877415},
 'derived': {'bit_efficency': 0.16208999866144166,
             'bit_redundancy': 404.5240956712259,
             'ideal_bit_efficency': 0.1916303916237789,
             'ideal_bit_redundancy': 330.1025128391366,
             'ideal_packet_efficiency': 0.9581519581188945,
             'ideal_packet_redundancy': 66.02050256782731,
             'ideal_time_to_recover': 81.67118343358214,
             'ideal_transmitted_bits': 408.3559171679107,
             'packet_efficiency': 0.8104499933072082,
             'packet_redundancy': 80.90481913424517,
             'permeability': 0.8458470354726778,
             'saturation': 15.65068086575483,
             'transmitted_bits': 482.77750000000003}}
Found implementation: anfext_fullrank2
  Fi

benchmark: 100%|██████████| 10000/10000 [12:31<00:00, 13.30it/s]

...

{'basic': {'expected_sample_burst_size': 2.998868425098141,
           'expected_sample_data_size': 16.5073385960547,
           'expected_time_to_recover': 127.7916151372743,
           'packet_size': 5,
           'payload_bits': 148.25340432877417},
 'derived': {'bit_efficency': 0.23202368037921692,
             'bit_redundancy': 490.70467135759736,
             'ideal_bit_efficency': 0.27417514440325996,
             'ideal_bit_redundancy': 392.4717757434257,
             'ideal_packet_efficiency': 1.3708757220162997,
             'ideal_packet_redundancy': 78.49435514868516,
             'ideal_time_to_recover': 108.14503601444,
             'ideal_transmitted_bits': 540.7251800721999,
             'packet_efficiency': 1.1601184018960846,
             'packet_redundancy': 98.14093427151947,
             'permeability': 0.8462608121688588,
             'saturation': 29.650680865754833,
             'transmitted_bits': 638.9580756863716}}
Found implementation: shuffled_binary_fullra

{'basic': {'expected_sample_burst_size': 3.0026418026418025,
           'expected_sample_data_size': 16.45827505827506,
           'expected_time_to_recover': 14.163100000000002,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3283660262057621,
             'bit_redundancy': 47.56209567122586,
             'ideal_bit_efficency': 0.3882730063334919,
             'ideal_bit_redundancy': 36.635910533360644,
             'ideal_packet_efficiency': 1.9413650316674596,
             'ideal_packet_redundancy': 7.327182106672128,
             'ideal_time_to_recover': 11.977862972426959,
             'ideal_transmitted_bits': 59.889314862134796,
             'packet_efficiency': 1.6418301310288108,
             'packet_redundancy': 9.512419134245171,
             'permeability': 0.8457091295286312,
             'saturation': 4.650680865754831,
             'transmitted_bits': 70.81550000000001}}
Found implementation: simple_matrix
  No

{'basic': {'expected_sample_burst_size': 2.993139238279532,
           'expected_sample_data_size': 16.47849415076084,
           'expected_time_to_recover': 34.083800000000004,
           'packet_size': 5,
           'payload_bits': 25.0},
 'derived': {'bit_efficency': 0.14669725793485466,
             'bit_redundancy': 145.419,
             'ideal_bit_efficency': 0.17334321932281038,
             'ideal_bit_redundancy': 119.22254356222302,
             'ideal_packet_efficiency': 0.8667160966140518,
             'ideal_packet_redundancy': 23.844508712444608,
             'ideal_time_to_recover': 28.844508712444608,
             'ideal_transmitted_bits': 144.22254356222302,
             'packet_efficiency': 0.7334862896742733,
             'packet_redundancy': 29.083800000000004,
             'permeability': 0.8462820669187299,
             'saturation': 5.0,
             'transmitted_bits': 170.419}}
Found implementation: single_binary_fullrank
  No changes. Using cached benchmark.


{'basic': {'expected_sample_burst_size': 2.9715212404081957,
           'expected_sample_data_size': 16.502254568467684,
           'expected_time_to_recover': 13.3366,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3487156296023597,
             'bit_redundancy': 43.429595671225854,
             'ideal_bit_efficency': 0.411508013268875,
             'ideal_bit_redundancy': 33.25437578480683,
             'ideal_packet_efficiency': 2.057540066344375,
             'ideal_packet_redundancy': 6.650875156961366,
             'ideal_time_to_recover': 11.301556022716197,
             'ideal_transmitted_bits': 56.507780113580985,
             'packet_efficiency': 1.7435781480117984,
             'packet_redundancy': 8.68591913424517,
             'permeability': 0.8474090864775277,
             'saturation': 4.650680865754831,
             'transmitted_bits': 66.683}}
Found implementation: single_binary_fullrank2
  No changes. Usin

{'basic': {'expected_sample_burst_size': 3.006353744738305,
           'expected_sample_data_size': 16.588197919148598,
           'expected_time_to_recover': 13.8138,
           'packet_size': 5,
           'payload_bits': 23.253404328774153},
 'derived': {'bit_efficency': 0.3366691906466599,
             'bit_redundancy': 45.81559567122585,
             'ideal_bit_efficency': 0.3976852628548549,
             'ideal_bit_redundancy': 35.2184740653243,
             'ideal_packet_efficiency': 1.9884263142742742,
             'ideal_packet_redundancy': 7.04369481306486,
             'ideal_time_to_recover': 11.694375678819691,
             'ideal_transmitted_bits': 58.471878394098454,
             'packet_efficiency': 1.6833459532332995,
             'packet_redundancy': 9.16311913424517,
             'permeability': 0.846571955495207,
             'saturation': 4.650680865754831,
             'transmitted_bits': 69.069}}
